In [ ]:
!pip install requests

Check if Ollama is running and show available models

In [27]:
import requests

def check_ollama_status():
    try:
        resp = requests.get("http://localhost:11434/api/tags", timeout=3)
        if resp.status_code == 200:
            models = [m['name'] for m in resp.json()['models']]
            print("Ollama is running locally.")
            print("Available models:", ', '.join(models) if models else "None yet (run 'ollama pull llama3' in your terminal).")
            return models
        else:
            print("Ollama responded, but not OK.")
    except Exception as e:
        print("Ollama is NOT running. Start it with the 'Ollama' app or via terminal: 'ollama serve'")
        raise

available_models = check_ollama_status()


Ollama is running locally.
Available models: llama3:latest, phi3:latest


Choose model to run and create the input text with data to anonymize

In [28]:
MODEL_NAME = "llama3:latest"

INPUT_TEXT = """
Jane Doe lives at 123 Main St, Springfield. Her phone number is (555) 123-4567. She works at Acme Corp.
"""

print(f"Using model: {MODEL_NAME}")

Using model: llama3:latest


Create the prompt with instructions and text to anonymize

In [29]:
prompt = f"""
Please anonymize the following conversation by replacing all names, addresses, phone numbers, and any other identifying information with generic placeholders. Output only the anonymized text.

TEXT TO ANONYMIZE:
{INPUT_TEXT}
"""


Send the prompt to Ollama and get the anonymized response

In [30]:
def anonymize_text(prompt, model):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "system": "You are an expert anonymizer. Never reveal real names or personal identifiable information.",
        "prompt": prompt,
        "stream": False
    }
    try:
        resp = requests.post(url, json=payload)
        result = resp.json()
        if 'response' in result:
            return result['response'].strip()
        else:
            return f"Error: {result}"
    except Exception as e:
        return f"Error contacting Ollama: {e}"

anonymized_output = anonymize_text(prompt, MODEL_NAME)


Display the result

In [31]:
# Cell 6: Display the result
from IPython.display import display, Markdown

print("Anonymized output:")
display(Markdown(f"```\n{anonymized_output}\n```"))


Anonymized output:


```
Here is the anonymized text:

[Person] lives at [Address]. Their phone number is ([Phone Number]). They work at [Company].
```